In [15]:
import pandas as pd
import requests
import re
import numpy as np
from sanky import*

In [10]:
df = pd.read_csv("food_production.csv", index_col=0)
df = df.reset_index()
del df['index']
df


,Title,DOI,Link,Abstract,Author Keywords,Index Keywords,FSC_stage
0,machine learning for predicting field soil moi...,10.3390/soilsystems5040057,https://www.scopus.com/inward/record.uri?eid=2...,precise soil moisture prediction is important ...,accumulated local effects random forest regres...,blank,food production
1,machine learning approach to simulate soil co2...,10.3390/agronomy12010197,https://www.scopus.com/inward/record.uri?eid=2...,with the growing number of datasets to describ...,gracenet gradient boosted regression greenhous...,blank,food production
2,machine learning techniques for iot data analy...,10.1002/9781119740780.ch3,https://www.scopus.com/inward/record.uri?eid=2...,tremendous advancements and innovations in har...,data analytics internet of things machine lear...,blank,food production
3,yield response of different rice ecotypes to m...,10.1155/2022/5305353,https://www.scopus.com/inward/record.uri?eid=2...,the food security of more than half of the wor...,blank,adaptive boosting food supply forecasting lime...,food production
4,a novel deep learning method for detection and...,10.1007/s40747-021-00536-1,https://www.scopus.com/inward/record.uri?eid=2...,the agricultural production rate plays a pivot...,agriculture centernet deep learning densenet p...,blank,food production
...,...,...,...,...,...,...,...
204,42nd sgai international conference on innovati...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 31 papers the special ...,blank,blank,food production
205,15th multi-disciplinary international conferen...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 19 papers the special ...,blank,blank,food production
206,aci 2022 - proceedings of the workshop on adva...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 38 papers the topics d...,blank,blank,food production
207,1st international conference on smart and sust...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 12 papers the special ...,blank,blank,food production


In [11]:
fps = {
    "Agriculture": ["crop rotation", "irrigation", "fertilizer", "harvesting", "cultivation", "planting", "soil health", "pest control", "greenhouse gases", "precision agriculture", "agricultural technology", "food security", "crop yield", "farming"],
    "Livestock": ["breeding", "feed", "grazing", "milk production", "meat processing", "animal welfare", "pasture management", "animal health", "genetics", "animal nutrition", "animal behavior", "meat quality"],
    "Fisheries": ["aquaculture", "fish farming", "wild-caught", "sustainable fishing", "fish processing", "fish market", "ocean health", "fish habitats", "marine conservation", "fisheries management", "bycatch reduction", "traceability"]
}


In [12]:
weights = {'Title': 0.4, 'Author Keywords': 0.3, 'Index Keywords': 0.2, 'Abstract':0.1}

# Create a function to count the number of times the words appear in a row
def count_words(row):
    counts = {}
    for key in list(fps.keys()):
        counts[key] = 0
        for word in fps[key]:
            for col in row.index:
                if col not in ['Authors', 'DOI', 'Link'] and word.lower() in re.findall(r'\b\w+\b',row[col]):
                    count = counts.get(key, 0)
                    counts[key] = count + weights.get(col, 1) if count > 0 else weights.get(col, 1)
    max_word = max(counts, key=counts.get) if any(counts.values()) else np.nan                    
    return pd.Series(counts), max_word

# Apply the function to each row of the dataframe
word_counts, max_words = zip(*df.apply(count_words, axis=1))

# Create a new dataframe with the word counts as columns
words_count_df = pd.DataFrame(list(word_counts), columns=list(fps.keys()))

# Add the max_word column to the original dataframe
df['Food production sector'] = max_words

# Concatenate the two dataframes
result_df = pd.concat([df, words_count_df], axis=1)
result_df

,Title,DOI,Link,Abstract,Author Keywords,Index Keywords,FSC_stage,Food production sector,Agriculture,Livestock,Fisheries
0,machine learning for predicting field soil moi...,10.3390/soilsystems5040057,https://www.scopus.com/inward/record.uri?eid=2...,precise soil moisture prediction is important ...,accumulated local effects random forest regres...,blank,food production,NaN,0.0,0.0,0.0
1,machine learning approach to simulate soil co2...,10.3390/agronomy12010197,https://www.scopus.com/inward/record.uri?eid=2...,with the growing number of datasets to describ...,gracenet gradient boosted regression greenhous...,blank,food production,NaN,0.0,0.0,0.0
2,machine learning techniques for iot data analy...,10.1002/9781119740780.ch3,https://www.scopus.com/inward/record.uri?eid=2...,tremendous advancements and innovations in har...,data analytics internet of things machine lear...,blank,food production,NaN,0.0,0.0,0.0
3,yield response of different rice ecotypes to m...,10.1155/2022/5305353,https://www.scopus.com/inward/record.uri?eid=2...,the food security of more than half of the wor...,blank,adaptive boosting food supply forecasting lime...,food production,NaN,0.0,0.0,0.0
4,a novel deep learning method for detection and...,10.1007/s40747-021-00536-1,https://www.scopus.com/inward/record.uri?eid=2...,the agricultural production rate plays a pivot...,agriculture centernet deep learning densenet p...,blank,food production,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
204,42nd sgai international conference on innovati...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 31 papers the special ...,blank,blank,food production,NaN,0.0,0.0,0.0
205,15th multi-disciplinary international conferen...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 19 papers the special ...,blank,blank,food production,Livestock,0.0,0.1,0.0
206,aci 2022 - proceedings of the workshop on adva...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 38 papers the topics d...,blank,blank,food production,NaN,0.0,0.0,0.0
207,1st international conference on smart and sust...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 12 papers the special ...,blank,blank,food production,Agriculture,0.2,0.0,0.0


In [13]:
df_res = result_df[result_df['Food production sector'].notna()]
#df_res = df_res.drop(["food production","food processing","food distribution","food retail"], axis= 1)
df_res

,Title,DOI,Link,Abstract,Author Keywords,Index Keywords,FSC_stage,Food production sector,Agriculture,Livestock,Fisheries
8,a taxonomy of food supply chain problems from ...,10.3390/s21206910,https://www.scopus.com/inward/record.uri?eid=2...,in the last few years the internet of things a...,agriculture computational intelligence deep le...,complex networks deep learning fish food suppl...,food production,Agriculture,0.6,0.0,0.0
10,stride-based cyber security threat modeling fo...,10.1109/STI53101.2021.9732597,https://www.scopus.com/inward/record.uri?eid=2...,the concept of traditional farming is changing...,cyber security internet of things iot precisio...,artificial intelligence decision support syste...,food production,Agriculture,0.1,0.0,0.0
12,agritrust—a trust management approach for smar...,10.3390/s20216174,https://www.scopus.com/inward/record.uri?eid=2...,internet of things iot provides a diverse plat...,digital innovation internet of agriculture thi...,agricultural robots agriculture internet of th...,food production,Agriculture,0.3,0.0,0.0
21,improved real-time semantic segmentation netwo...,10.3389/fpls.2022.898131,https://www.scopus.com/inward/record.uri?eid=2...,field crops are generally planted in rows to i...,crop rows detection navigation path recognitio...,blank,food production,Agriculture,0.1,0.0,0.0
28,machine learning based smart crop recommender ...,10.1109/ICCCIS56430.2022.10037678,https://www.scopus.com/inward/record.uri?eid=2...,commercialization of agriculture industry rais...,agriculture crop recommendation machine learni...,cultivation decision trees forecasting learnin...,food production,Agriculture,0.2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
195,integrating machine learning interpretation me...,10.1039/d2nr01904c,https://www.scopus.com/inward/record.uri?eid=2...,seed priming by nanoparticles is an environmen...,blank,cultivation ii-vi semiconductors ionic strengt...,food production,Agriculture,0.2,0.0,0.0
197,a survey on deep learning applications in whea...,10.1016/j.asoc.2022.109761,https://www.scopus.com/inward/record.uri?eid=2...,precision farming has become a hot research to...,agriculture convolutional neural networks deep...,agriculture convolutional neural networks cost...,food production,Agriculture,0.3,0.0,0.0
202,asynchrony between urban expansion and water e...,10.3389/fenvs.2022.1018408,https://www.scopus.com/inward/record.uri?eid=2...,the asynchrony of urban expansion and urban wa...,nitrogen and phosphorus small water bodies spa...,blank,food production,Agriculture,0.1,0.0,0.0
205,15th multi-disciplinary international conferen...,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 19 papers the special ...,blank,blank,food production,Livestock,0.0,0.1,0.0


In [14]:
result_df['Food production sector'].value_counts()

Agriculture    55
Livestock      16
Fisheries       3
Name: Food production sector, dtype: int64

In [ ]:
fig = genSankey(df_res1,cat_cols=['FSC_stage','AI_models' ],value_cols='count',title='Taxonomy AI in Food supply chain')
plotly.offline.plot(fig, validate=False)